In [ ]:
import pandas as pd
import numpy as np

In [ ]:
meta_data = pd.read_csv("arquivos_extraidos/2002/2002/INMET_CO_GO_A002_GOIANIA_01-01-2002_A_31-12-2002.CSV", header=None,
        sep=":;",
        encoding="latin-1",
        nrows=8,
        decimal=","
)
meta_data = pd.DataFrame({
    0: ["REGIÃO", "UF", "ESTAÇÃO", "CODIGO (WMO)", "LATITUDE", "LONGITUDE", "ALTITUDE", "DATA DE FUNDAÇÃO (YYYY-MM-DD)"],
    1: ["CO", "GO", "GOIANIA", "A002", -16.64277777, -49.21999999, 770, "2001-05-29"]
})

meta_data = meta_data.set_index(0).T




In [ ]:
dicionario_abreviacoes = {
    "DATA": "Data da medição",
    "HORA": "Hora da medição (UTC)",
    "PRECIP_TOTAL": "Precipitação total horária (mm)",
    "PRESSAO_ATM": "Pressão atmosférica ao nível da estação (mB)",
    "PRESSAO_MAX": "Pressão atmosférica máxima na hora anterior (mB)",
    "PRESSAO_MIN": "Pressão atmosférica mínima na hora anterior (mB)",
    "RADIACAO": "Radiação global (KJ/m²)",
    "TEMP_AR": "Temperatura do ar - bulbo seco (°C)",
    "TEMP_ORVALHO": "Temperatura do ponto de orvalho (°C)",
    "TEMP_MAX": "Temperatura máxima na hora anterior (°C)",
    "TEMP_MIN": "Temperatura mínima na hora anterior (°C)",
    "TEMP_ORV_MAX": "Temperatura máxima do orvalho na hora anterior (°C)",
    "TEMP_ORV_MIN": "Temperatura mínima do orvalho na hora anterior (°C)",
    "UMID_MAX": "Umidade relativa máxima na hora anterior (%)",
    "UMID_MIN": "Umidade relativa mínima na hora anterior (%)",
    "UMID_AR": "Umidade relativa do ar (horária) (%)",
    "VENTO_DIR": "Direção do vento (gr) (°)",
    "VENTO_RAJADA": "Rajada máxima do vento (m/s)",
    "VENTO_VEL": "Velocidade horária do vento (m/s)",
    "UNNAMED": "Coluna não nomeada ou irrelevante"
}

In [ ]:
main_data = pd.read_csv(
    "arquivos_extraidos/2004/2004/INMET_CO_MT_A907_RONDONOPOLIS_01-01-2004_A_31-12-2004.CSV", 
    sep=";",
    encoding="latin-1",
    decimal=",",
    skiprows=8
)

names_columns = [
    "DATA", "HORA", "PRECIP_TOTAL", "PRESSAO_ATM", 
    "PRESSAO_MAX", "PRESSAO_MIN", "RADIACAO", 
    "TEMP_AR", "TEMP_ORVALHO", "TEMP_MAX", 
    "TEMP_MIN", "TEMP_ORV_MAX", "TEMP_ORV_MIN", 
    "UMID_MAX", "UMID_MIN", "UMID_AR", 
    "VENTO_DIR", "VENTO_RAJADA", "VENTO_VEL", 
    "UNNAMED"
]

main_data.columns = names_columns
main_data.replace(-9999, np.nan, inplace=True)
main_data["DATA_HORA"] = pd.to_datetime(main_data["DATA"] + " " + main_data["HORA"], format="%Y-%m-%d %H:%M")
main_data = main_data.drop(columns=["DATA", "HORA"])
main_data = main_data.drop(columns=["UNNAMED"])
main_data = main_data[["DATA_HORA"] + [col for col in main_data.columns if col != "DATA_HORA"]]

for column in meta_data.columns:
    main_data[column] = meta_data[column].iloc[0]

In [ ]:
def load_weather_data(file_path, meta_data):
        main_data = pd.read_csv(
        file_path,
        sep=";",
        encoding="latin-1",
        decimal=",",
        skiprows=8
    )
    

In [ ]:
import logging
import os
import pandas as pd
from collect_link import collect_links
from download_files import download_files
from extract_files import unzip_files
from treatment_files import load_metadata, load_weather_data, result

try:
    pasta_tratada = "arquivos_tratados"
    os.makedirs(pasta_tratada, exist_ok=True)

    
    links = collect_links()
    if not links:
        logging.warning("Nenhum link encontrado.")

   
    download_files(links)

    unzip_files()

    meta_data = load_metadata()
    if meta_data is None:
        logging.error("Erro ao carregar metadados.")
        
    file_path = (r"arquivos_extraidos/2002/2002/INMET_CO_GO_A002_GOIANIA_01-01-2002_A_31-12-2002.CSV")    
    load_weather_data(file_path,meta_data)
    
    
    result_df = result(meta_data)
    logging.info(f"Resultado final: {result_df}")

except Exception as e:
    logging.error(f"Erro no processamento: {e}")


In [ ]:
print(meta_data)

In [ ]:
result = pd.concat([meta_data, main_data], axis=0, ignore_index=True,)

In [ ]:
result